## CIS 9
## Natural Language Processing with NLTK



Reading
<br>Natural Language Processing with Python: Learning to classify text, up to section 1.3

_Natural language_ is the language that humans use to communicate with each other. It's the language we use in our conversation or when we read and write texts, as opposed to computer languages.

_Natural Language Processing (NLP)_ or _Computational Linguistics_ is a field that uses computers to analyze, understand, and generate natural human language. NLP is used for tasks from counting word frequencies to allowing computers to interpret human commands to translating languages. NLP uses machine learning algorithms to build probabilistic model about a language.

Import libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import random
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import nltk

### A. Common NLTK tools for pre-processing text
<br>Since texts are human readable words while machines only work with numbers, often in NLP we need to transform the text into meaningful numbers and possibly reduce the number of words in the text before we can use machine learning algorithms with the text. This is called pre-processing the text, and it's similar to the step in clasic machine learning where we change a feature description (with words) into a corresponding sequence of numbers, or when we use dimensional reduction to remove the number of irrelevant or duplicate features. 

1. __Tokenize__: separate the text into words or sentences. A sentence, paragraph, or an entire essay or writing is  made of words that give meaning to the text, so the dataset in NLP is generally the words in the text. A _tokenizer_ makes it simple to separate the text into words.

NTLK has many packages and not all available packages are downloaded by default, so we need to download and install the tokenizer the first time we use it. 
<br>(If you forget to comment out the download line of code after the first time, don't worry, the package manager is smart enough to figure out that you've downloaded it already.)

In [ ]:
#nltk.download('punkt')     # only need to run this one time to download and install the package

In [ ]:
# Sample line of text from python.org
s = '''Python is a programming language that lets you work quickly and integrate systems more effectively.
It's a programmer friendly language.'''  # adding a second line of text

In [ ]:
from nltk.tokenize import word_tokenize   # separate text into words

words = word_tokenize(s)
print(words)

# a. How does this compare with Python string split() method?                 

In [ ]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
only_words = tokenizer.tokenize(s)
print(only_words)

# b. How is this list of words diferent from the previous list?                

2. __Frequency distribution__: count of occurrences of words. Sometimes looking at the words that are most common in the text can give us some general idea about the text: is it casual or formal, what could be the main topics, etc.

In [ ]:
freqD = nltk.FreqDist(only_words)
for key,val in freqD.items():
    print(str(key) + ':' + str(val))
print(len(freqD))

# We know enough Python to easily write the code for FreqDist, but it's nice to just call it

In [ ]:
freqD.plot(20)    

# What happens when the input argument is 10?                                  

3. __Stop words__: words that appear often in the text but don't necessarily help us find the meaning of the text. Example stop words are 'an', 'the', 'to', 'that'.
<br>Sometimes it's useful to remove the stop words because it can reduce the size of the datasets, making it faster to train the algorithm. But sometimes it's a bad idea to remove stop words if they actually have meaning in the text. For classification problems it's generally fine to remove the stop words.
<br>The list of stop words are different for different NLP tools, NLTK has different stop words than other tools.

We need to download and install the NLTK package for stopwords

In [ ]:
#nltk.download('stopwords')      # only need to run this one time to download and install the package

In [ ]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
print(stop_words)

In [ ]:
# Create a list of words from only_words above, but with no stop words           

print("Words:",only_words)
print()
print("Without stop words:",)

4. __Stemming__: removing parts of a word to get to its _stem_ or root. This can be useful to reduce the number of words we need to analyze. For example: 'talk', 'talking', 'talked', 'talks' all have the same stem 'talk' and perhaps can be grouped together. That way they are only counted once, or that way we can count their frequency as a group.

In [ ]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
stemmed=[stemmer.stem(w) for w in only_words]
print("Words:",only_words)
print()
print("Stems:",stemmed)

5. __Part of speech (POS) tagging__: find the part of speech for each word. Sometime it's important to identify the part of speech of a word because it can make a difference in the meaning of a word. For example, 'drive' can be a verb in 'They drive fast', or 'drive' can be a noun in 'The drive was long and hot'.

We need to download and install the NLTK package for tagging POS

In [ ]:
#nltk.download('averaged_perceptron_tagger')    # only need to run this one time to download and install the package
# The name perceptron hints at the fact that the tagger is done with a neural network. 
# More on this later.

In [ ]:
wordTags = nltk.pos_tag(only_words)   
print(wordTags)

The list of tags and their meanings are [here](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html)

In [ ]:
# From wordTags above, find all the verbs


---

### B. Text classification

We first explore NLTK's own machine learning algorithm, the Naive Bayes model, for text classification. Naive Bayes is one of the most common algorithm to use with text because it can be fairly accurate with the prediction without needing a large dataset. It was initially introduced for text classification tasks and is still used as a benchmark.

__Predicting names__
<br>We start with a simple learning algorithm that works with a list of people's first names and genders, then we train the algorithm to predict the gender of a person, given the person's name. In this simple case the names are one-word features and capitalized correctly, so we don't need to do any pre-processing of data. We don't need to:
- make the name case insensitive by converting them to lowercase
- tokenize the names since they are already single words
- the names are proper nouns so no need to remove stop words or reduce them to stems or determine the part of speech

In the English language, male first names and female first names tend to have different endings or different last letters. Names ending in a, e and i are likely to be female, while names ending in k, o, r, s and t are likely to be male. We will build a classifier to check the accuracy of predicting whether a name is male or female based on the last letter of the name.

6. Download and install the names library from NLTK 

In [ ]:
#nltk.download('names')    # (only need to run this one time to download and install

7. Get a list of male and female names from the NLTK library, assign labels to them, and randomize the order

In [ ]:
from nltk.corpus import names
 
allnames = ([(name, 'male') for name in names.words('male.txt')] +
         [(name, 'female') for name in names.words('female.txt')])
print(len(allnames))
print(allnames[:2], allnames[-2:])   # alphabetical order
random.shuffle(allnames)          # random order
print(allnames[:2], allnames[-2:])

8. The ntlk Naive Bayes model is trained with a _feature set_, which is a list of tuples, where each tuple is the input feature and corresponding output label.
<br>An input feature is a dictionary where the key is a short text description, and the value is a single boolean or integer (character). The labels can be text strings.

In [ ]:
def getFeature(word): 
    return {'lett': word[-1]}  # use last letter as feature

# Using the function getFeature above, which returns a dictionary, create the list 
# which is the feature set (the feature set is described in the cell above).
# Example of one element of the feature set: 
# If the name is ('Alane', 'female'), then the corresponding element in the 
# feature set is: ({'lett':'e'},'female')



9. Split the dataset into a training and testing set, then use the training set to train the classifer

In [ ]:
from nltk.classify import NaiveBayesClassifier

train_set, test_set = train_test_split(featureSet,test_size=0.2)
print(len(train_set), len(test_set))
classifier = nltk.NaiveBayesClassifier.train(train_set)

10. Check the classifier accuracy

In [ ]:
nltk.classify.accuracy(classifier, test_set)

In [ ]:
classifier.show_most_informative_features(10)

__Bag of Words model__
<br>In this next example we work with a larger problem where we train a model to predict whether SMS messages are spam or not. Even though SMS messages are generally short, they are sentences with multiple words. This means the dataset is large and will need to be pre-processed before they are converted to numbers to work with a ML model.

A bag of words model is the simplest way to analyze data. It reduces the text to a container of words, without considering the order of the words or how the words are put together in a sentence (such as, is it part of a question or a statement or an exclamation). Only the number of occurrences or frequency of each word is counted.

Bag of words model is useful in classification to determine whether the text is a particular type of text.

11. Just like "Hello world" is the intro to programming, and "Ice cream sales" is the intro to machine learning, "Ham vs. spam" is the intro to NLP.
<br>We will try to classify whether an SMS is spam or ham (not spam). We use the input file _spam_short.csv_ from [source](https://www.kaggle.com/uciml/sms-spam-collection-dataset) that has been shortened to the first 1000 entries.

In [ ]:
inputData = pd.read_csv('spam_short.csv', encoding='ISO-8859-1')   # Unicode characters
inputData.head()

12. Remove the extra columns with NaN and change the column labels from 'v1' and 'v2' to 'label' and 'text'.
<br>Print the DataFrame

13. Create the y labels from the 'label' column and convert 'ham' to 0 and 'spam' to 1
<br>Show the size of y and the first few lines of y.

14. Create the X Dataframe from the 'text' column.
<br>Print the size of X and the first few lines of X

15. Use the pre-processing techniques in part A to reduce the X dataset:
- remove all numbers and punctuations (ony words with letters should remain)
- change all words to lowercase
- remove all stop words
- stem all words

In [ ]:
# EC: write code to do the steps in Q15. 
# You should end up with an X DataFrame, name it X_processed



16. Transform each word into a document vector, which is a way to give a score to each word, by using the _CountVectorizer_
<br>The CountVectorizer counts the frequency of each word, and returns a vector of numbers (features) that can be used by a machine learning model

In [ ]:
# Example of how a CountVectorizer works

from sklearn.feature_extraction.text import CountVectorizer

text = ["The quick brown fox jumps over the lazy dog"]  # classic typing exercise
vectorizer = CountVectorizer()
vectorizer.fit(text)      
print("Frequency count:", vectorizer.vocabulary_)

vector = vectorizer.transform(text)   # output vector to be used by ML algorithms
print(vector.shape)
print(vector)
print(vectorizer.get_feature_names())

17. Now we apply CountVectorizer to our X_processed dataset

In [ ]:
vect = CountVectorizer()
vect.fit(X_processed[0])
X_vectors = vect.transform(X_processed[0])
print(X_vectors[0])
X_vectors.shape

18. Split the X_vectors and y datasets into training and testing sets and print the size of the sets.

In [ ]:

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

19. Recall that Multinomial Naive Bayes take each feature and its frequency into account to do output prediction, so it fits very well with the X_vectors. We train the classifier and test it.

20. Check the accuracy of the model and show the confusion matrix.

In [ ]:
# accuracy
print(metrics.accuracy_score(y_test, y_pred))
metrics.confusion_matrix(y_test, y_pred, labels=[0,1])

NLP is a big and challenging field because natural languages are messy, there are as many exceptions as there are rules in human languages, and languages evolve quickly. But it's also a field that is growing in importance with the pervasive role of social media and the advances of smart devices that interact us by using our human languages. 

_Optional_ note: We saw that the POS tagger is created from a neural network or perceptron. For those who are interested in reading an example of creating a neural network to do POS tagging, see this [article](https://becominghuman.ai/part-of-speech-tagging-tutorial-with-the-keras-deep-learning-library-d7f93fa05537)
<br>You'll recognize the same steps of: cleaning / pre-processing data, splitting the data into training and testing sets, creating a neural network, training the model, and evaluate the accuracy.